# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
import pandas as pd
import numpy as np

import statsmodels.formula.api as smf
import patsy

In [2]:
df = pd.read_csv('previsao_de_renda.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

### I. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis. 

In [4]:
reg = smf.ols('''np.log(renda) ~ 
                sexo 
                + posse_de_veiculo 
                + qtd_filhos 
                + tipo_renda 
                + educacao 
                + estado_civil
                + tipo_residencia 
                + idade
                + tempo_emprego 
                + qt_pessoas_residencia''',
              data = df)

### 1) Utilizando os recursos do Patsy, coloque as variáveis qualitativas como dummies.

In [5]:
reg = smf.ols('''np.log(renda) ~ 
                C(sexo) 
                + C(posse_de_veiculo) 
                + C(qtd_filhos) 
                + C(tipo_renda) 
                + C(educacao)
                + C(estado_civil)
                + C(tipo_residencia)
                + idade
                + tempo_emprego 
                + C(qt_pessoas_residencia)''',
              data = df)

### 2) Mantenha sempre a categoria mais frequente como casela de referência

In [6]:
df.mode(axis=0).loc[0, ['sexo', 'posse_de_veiculo', 'posse_de_imovel', 'qtd_filhos',
       'tipo_renda', 'educacao', 'estado_civil', 'tipo_residencia',
       'qt_pessoas_residencia']]

sexo                               F
posse_de_veiculo               False
posse_de_imovel                 True
qtd_filhos                       0.0
tipo_renda               Assalariado
educacao                  Secundário
estado_civil                  Casado
tipo_residencia                 Casa
qt_pessoas_residencia            2.0
Name: 0, dtype: object

In [7]:
reg = smf.ols('''np.log(renda) ~ 
                C(sexo, Treatment("F")) 
                + C(posse_de_veiculo, Treatment(False))
                + C(posse_de_imovel, Treatment(True))
                + C(qtd_filhos, Treatment(0))
                + C(tipo_renda, Treatment("Assalariado"))
                + C(educacao, Treatment ("Secundário"))
                + C(estado_civil, Treatment ("Casado"))
                + C(tipo_residencia, Treatment ("Casa"))
                + idade
                + tempo_emprego 
                + C(qt_pessoas_residencia, Treatment(2))''',
              data = df).fit()

### 3) Avalie os parâmetros e veja se parecem fazer sentido prático.

In [8]:
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.358
Model:                            OLS   Adj. R-squared:                  0.357
Method:                 Least Squares   F-statistic:                     203.5
Date:                Tue, 19 Mar 2024   Prob (F-statistic):               0.00
Time:                        11:53:07   Log-Likelihood:                -13559.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12392   BIC:                         2.745e+04
Df Model:                          34                                         
Covariance Type:            nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
Intercept                                                       7.1759      0.039    185.454      0.000       7.100       7.252
C(sexo, Treatment("F"))[T.M]                                    0.7878      0.015     53.664      0.000       0.759       0.817
C(posse_de_veiculo, Treatment(False))[T.True]                   0.0436      0.014      3.079      0.002       0.016       0.071
C(posse_de_imovel, Treatment(True))[T.False]                   -0.0828      0.014     -5.907      0.000      -0.110      -0.055
C(qtd_filhos, Treatment(0))[T.1]                               -0.2501      0.112     -2.239      0.025      -0.469      -0.031
C(qtd_filhos, Treatment(0))[T.2]                               -0.5697      0.223     -2.559      0.011      -1.006      -0.133
C(qtd_filhos, Treatment(0))[T.3]                               -0.5560      0.486     -1.144      0.253      -1.509       0.397
C(qtd_filhos, Treatment(0))[T.4]                                0.2091      0.429      0.488      0.626      -0.631       1.049
C(qtd_filhos, Treatment(0))[T.5]                                0.1385      0.410      0.338      0.736      -0.666       0.943
C(qtd_filhos, Treatment(0))[T.7]                                0.0221      0.162      0.137      0.891      -0.295       0.339
C(qtd_filhos, Treatment(0))[T.14]                              -0.4630      0.261     -1.772      0.076      -0.975       0.049
C(tipo_renda, Treatment("Assalariado"))[T.Bolsista]             0.2259      0.241      0.937      0.349      -0.247       0.699
C(tipo_renda, Treatment("Assalariado"))[T.Empresário]           0.1547      0.015     10.359      0.000       0.125       0.184
C(tipo_renda, Treatment("Assalariado"))[T.Pensionista]         -0.2891      0.243     -1.190      0.234      -0.765       0.187
C(tipo_renda, Treatment("Assalariado"))[T.Servidor público]     0.0551      0.022      2.476      0.013       0.011       0.099
C(educacao, Treatment("Secundário"))[T.Primário]                0.0130      0.072      0.181      0.856      -0.128       0.154
C(educacao, Treatment("Secundário"))[T.Pós graduação]           0.1168      0.142      0.821      0.412      -0.162       0.396
C(educacao, Treatment("Secundário"))[T.Superior completo]       0.1069      0.014      7.678      0.000       0.080       0.134
C(educacao, Treatment("Secundário"))[T.Superior incompleto]    -0.0290      0.032     -0.897      0.370      -0.092       0.034
C(estado_civil, Treatment("Casado"))[T.Separado]                0.3176      0.112      2.831      0.005       0.098       0.537
C(estado_civil, Treatment("Casado"))[T.Solteiro]                0.2556      0.110      2.319      0.020       0.040       0.472
C(estado_ci

**Resposta:** Vários parâmetros de modo geral não fazem sentido prático, pois temos muitos p-values acima do valor de 0.05.

### II. Remova a variável menos significante e analise

**Resposta:** Irei remover "tipo_residência", pois todos os indicadores p-value dessa variável estão acima de 0.05.

In [9]:
reg = smf.ols('''np.log(renda) ~ 
                C(sexo, Treatment("F")) 
                + C(posse_de_veiculo, Treatment(False))
                + C(posse_de_imovel, Treatment(True))
                + C(qtd_filhos, Treatment(0))
                + C(tipo_renda, Treatment("Assalariado"))
                + C(educacao, Treatment ("Secundário"))
                + C(estado_civil, Treatment ("Casado"))
                + idade
                + tempo_emprego 
                + C(qt_pessoas_residencia, Treatment(2))''',
              data = df).fit()

### 1) Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.

In [10]:
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.358
Model:                            OLS   Adj. R-squared:                  0.357
Method:                 Least Squares   F-statistic:                     238.5
Date:                Tue, 19 Mar 2024   Prob (F-statistic):               0.00
Time:                        11:57:39   Log-Likelihood:                -13561.
No. Observations:               12427   AIC:                         2.718e+04
Df Residuals:                   12397   BIC:                         2.741e+04
Df Model:                          29                                         
Covariance Type:            nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
Intercept                                                       7.1790      0.038    187.436      0.000       7.104       7.254
C(sexo, Treatment("F"))[T.M]                                    0.7896      0.015     53.937      0.000       0.761       0.818
C(posse_de_veiculo, Treatment(False))[T.True]                   0.0436      0.014      3.084      0.002       0.016       0.071
C(posse_de_imovel, Treatment(True))[T.False]                   -0.0817      0.014     -5.943      0.000      -0.109      -0.055
C(qtd_filhos, Treatment(0))[T.1]                               -0.2492      0.112     -2.231      0.026      -0.468      -0.030
C(qtd_filhos, Treatment(0))[T.2]                               -0.5694      0.223     -2.558      0.011      -1.006      -0.133
C(qtd_filhos, Treatment(0))[T.3]                               -0.5605      0.486     -1.153      0.249      -1.513       0.392
C(qtd_filhos, Treatment(0))[T.4]                                0.1888      0.428      0.441      0.659      -0.650       1.028
C(qtd_filhos, Treatment(0))[T.5]                                0.1192      0.410      0.291      0.771      -0.684       0.922
C(qtd_filhos, Treatment(0))[T.7]                                0.0214      0.162      0.133      0.894      -0.295       0.338
C(qtd_filhos, Treatment(0))[T.14]                              -0.4639      0.261     -1.776      0.076      -0.976       0.048
C(tipo_renda, Treatment("Assalariado"))[T.Bolsista]             0.2271      0.241      0.942      0.346      -0.246       0.700
C(tipo_renda, Treatment("Assalariado"))[T.Empresário]           0.1551      0.015     10.401      0.000       0.126       0.184
C(tipo_renda, Treatment("Assalariado"))[T.Pensionista]         -0.2917      0.243     -1.202      0.230      -0.768       0.184
C(tipo_renda, Treatment("Assalariado"))[T.Servidor público]     0.0563      0.022      2.534      0.011       0.013       0.100
C(educacao, Treatment("Secundário"))[T.Primário]                0.0087      0.072      0.121      0.904      -0.132       0.149
C(educacao, Treatment("Secundário"))[T.Pós graduação]           0.1153      0.142      0.810      0.418      -0.164       0.394
C(educacao, Treatment("Secundário"))[T.Superior completo]       0.1066      0.014      7.674      0.000       0.079       0.134
C(educacao, Treatment("Secundário"))[T.Superior incompleto]    -0.0287      0.032     -0.891      0.373      -0.092       0.034
C(estado_civil, Treatment("Casado"))[T.Separado]                0.3182      0.112      2.837      0.005       0.098       0.538
C(estado_civil, Treatment("Casado"))[T.Solteiro]                0.2567      0.110      2.330      0.020       0.041       0.473
C(estado_ci

**Resposta:** O modelo melhorou pouquíssimo, tendo em vista que R² continua igual e que AIC diminuiu muito pouco.

### 2) Observe os parâmetros e veja se algum se alterou muito.

**Resposta:** Nenhum parâmetro sofreu grande alteração.


### III. Siga removendo as variáveis menos significantes, sempre que o p-value for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor.

In [11]:
reg = smf.ols('''np.log(renda) ~ 
                C(sexo, Treatment("F")) 
                + C(posse_de_veiculo, Treatment(False))
                + C(posse_de_imovel, Treatment(True))
                + idade
                + tempo_emprego ''',
              data = df).fit()

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.346
Model:                            OLS   Adj. R-squared:                  0.346
Method:                 Least Squares   F-statistic:                     1315.
Date:                Tue, 19 Mar 2024   Prob (F-statistic):               0.00
Time:                        11:59:00   Log-Likelihood:                -13676.
No. Observations:               12427   AIC:                         2.736e+04
Df Residuals:                   12421   BIC:                         2.741e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=================================================================================================================
                                                    coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
Intercept                                         7.3430      0.032    230.088      0.000       7.280       7.406
C(sexo, Treatment("F"))[T.M]                      0.7700      0.015     52.851      0.000       0.741       0.799
C(posse_de_veiculo, Treatment(False))[T.True]     0.0579      0.014      4.116      0.000       0.030       0.085
C(posse_de_imovel, Treatment(True))[T.False]     -0.0880      0.014     -6.379      0.000      -0.115      -0.061
idade                                             0.0044      0.001      5.871      0.000       0.003       0.006
tempo_emprego                                     0.0611      0.001     59.199      0.000       0.059       0.063
==============================================================================
Omnibus:                        1.248   Durbin-Watson:                   2.026
Prob(Omnibus):                  0.536   Jarque-Bera (JB):                1.221
Skew:                           0.022   Prob(JB):                        0.543
Kurtosis:                       3.019   Cond. No.                         211.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

**Resposta:** Removidas as variáveis de menor significância, estranhamente parece que o modelo piorou, pois tanto R² quanto R²-ajustado diminuiram quanto AIC aumentou.